### Update Building Scene Layers for Construction Monitoring

#### This Notebook helps to update existing building scene layer using new revit files to update construction progress of statino structures

#### Steps
#### A. Process new revit files (with new construction status)
1. Define Projection to PRS92
2. BIM to Geodatabase (file geodatabase)
3. Make Building Layers

#### B. Update Existing Building Scene Layer
##### For Each building component layer, repeat the follow:
1. Delete Field 'Status'
2. Join Field using new layer ([transfer field = 'Status'])

#### C. Delete temporary file geodatabase
1. Delete file geodatabase created during the process 'A'

In [30]:
import time, os, datetime, sys, logging, logging.handlers, shutil, traceback, re
from pathlib import Path
import arcpy

arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("WGS 1984 Web Mercator (auxiliary sphere)")
arcpy.env.geographicTransformations = "PRS_1992_To_WGS_1984_1"

home = Path.home()
directory = r'OneDrive - Oriental Consultants Global JV/Documents/ArcGIS/Projects/During-Construction_nscrexn2'
workspace = os.path.join(home, directory)

BuildingLayerName = "temporary_BIM_layers"

### A. Process new revit files

In [36]:
out_gdb_path = os.path.join(workspace,"During-Construction_nscrexn2.gdb")
out_dataset_name = "temporary_BIM_dataset"

In [37]:
# 2. BIM File to Geodatabase
## This saves all Revit models to enterprise geodatabase
RevitWD = os.path.join(home, "Dropbox/01-Railway/02-NSCR-Ex/01-N2/01-Basedata/09-Station_Structure/99-BIM Model") 
Revit1 = os.path.join(RevitWD,"N-01/NSCR-GCR-N01-APLSTN-MOD-ST-000001.rvt")
Revit2 = os.path.join(RevitWD, "N-01/NSCR-GCR-N01-CLPSTN-MOD-ST-000001.rvt")

inputRevit = [Revit1,Revit2]
# Choose enterprise geodatabase for Output Geodatabase
spatial_reference = "PRS_1992_Philippines_Zone_III"

arcpy.BIMFileToGeodatabase_conversion(inputRevit, out_gdb_path, out_dataset_name, spatial_reference)


<Result 'C:\\Users\\eiji.LAPTOP-KSD9P6CP\\OneDrive - Oriental Consultants Global JV/Documents/ArcGIS/Projects/During-Construction_nscrexn2\\During-Construction_nscrexn2.gdb\\temporary_BIM_dataset'>

In [38]:
# 3. Make Building Layer
out_dataset = os.path.join(out_gdb_path, out_dataset_name)
arcpy.MakeBuildingLayer_management(out_dataset,BuildingLayerName)

<Result 'temporary_BIM_layers'>

In [ ]:
# 4. Make sure to 'remove empty layer'

In [48]:
# 5. Specificy input Tables
buildingLayerNameArchitectural = BuildingLayerName + "\\" + "Architectural"
buildingLayerNameStructural = BuildingLayerName + "\\" + "Structural"

input_features = [buildingLayerNameStructural + "\\" + "StructuralColumns",
               buildingLayerNameStructural + "\\" + "StructuralFoundation",
               buildingLayerNameStructural + "\\" + "StructuralFraming",
               buildingLayerNameArchitectural + "\\" + "Columns",
               buildingLayerNameArchitectural + "\\" + "Floors",
               buildingLayerNameArchitectural + "\\" + "Walls"
              ]

basename_input = []
for feature in input_features:
    basename = os.path.basename(feature)
    basename_input.append(basename)

### B. Update Existing Building Scene Layer

In [40]:
# 1. Specify Existing Building Layers (to be updated)
existingBuildingLayer = 'N2_Station_Structure_practice'
existingBuildingLayerNameArchitectural = existingBuildingLayer + "\\" + "Architectural"
existingBuildingLayerNameStructural = existingBuildingLayer + "\\" + "Structural"

target_features = [existingBuildingLayerNameStructural + "\\" + "StructuralColumns",
               existingBuildingLayerNameStructural + "\\" + "StructuralFoundation",
               existingBuildingLayerNameStructural + "\\" + "StructuralFraming",
               existingBuildingLayerNameArchitectural + "\\" + "Columns",
               existingBuildingLayerNameArchitectural + "\\" + "Floors",
               existingBuildingLayerNameArchitectural + "\\" + "Walls"
              ]

In [50]:
# 2. Join Field
## Join updating new building layers to existing layers
## When starting to receive real revit models, make sure to use 'Status'

deleteTransferField = 'Family'
#deleteTransferField = 'Status'
joinField = 'ObjectId'


for feature in target_features:
    arcpy.DeleteField_management(feature, [deleteTransferField])
    
    # Get input_feature corresponding to target_feature
    basename = os.path.basename(feature)
    index = basename_input.index(basename)
    joinTable = input_features[index]
    
    # Join
    arcpy.management.JoinField(feature, joinField, joinTable, joinField, deleteTransferField)

### C. Delete temporary feature dataset